<a href="https://colab.research.google.com/github/parisakh4/Thesis/blob/main/TabDDPM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/

/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main


In [12]:
!cat requirements.txt


catboost==1.0.3
category-encoders==2.3.0
dython==0.5.1
icecream==2.1.2
libzero==0.0.8
numpy==1.21.4
optuna==2.10.1
pandas==1.3.4
pyarrow==6.0.0
rtdl==0.0.9
scikit-learn==1.0.2
scipy==1.7.2
skorch==0.11.0
tomli-w==0.4.0
tomli==1.2.2
tqdm==4.62.3

# smote
imbalanced-learn==0.7.0

# tvae
rdt==0.6.4 

In [13]:
# Replace the specific line using shell command
!sed -i 's/catboost==1.0.3/catboost==1.2.8/' requirements.txt


In [6]:
!grep catboost requirements.txt


catboost==1.2.8


In [17]:
requirements_txt = """
catboost==1.2.8
category-encoders==2.3.0
dython==0.5.1
icecream==2.1.2
libzero==0.0.8
numpy==1.24.4
optuna==2.10.1
pandas==1.5.3
pyarrow==10.0.1
rtdl==0.0.9
scikit-learn==1.2.2
scipy==1.10.1
skorch==0.11.0
tomli-w==0.4.0
tomli==2.0.1
tqdm==4.66.1

# smote
imbalanced-learn==0.10.1

# tvae
rdt==1.3.0
"""

with open("requirements_colab.txt", "w") as f:
    f.write(requirements_txt)


In [7]:
!pip install -r requirements_colab.txt



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.8/35.8 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78

In [4]:
 import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine_processed.csv')


In [5]:
from sklearn.model_selection import train_test_split

# 70% train, 30% temp
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# 15% val, 15% test from the remaining 30%
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [6]:
import os

output_dir = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine"
os.makedirs(output_dir, exist_ok=True)

# Save
train_df.to_csv(f"{output_dir}/train.csv", index=False)
val_df.to_csv(f"{output_dir}/val.csv", index=False)
test_df.to_csv(f"{output_dir}/test.csv", index=False)


In [43]:
config_text = """
seed = 0
parent_dir = "exp/wine/check"
real_data_path = "data/wine/"
model_type = "mlp"
num_numerical_features = 11
device = "cuda:0"

[model_params]
is_y_cond = false
d_in = 12
num_classes = 0

[model_params.rtdl_params]
d_layers = [256, 256]
dropout = 0.0

[diffusion_params]
num_timesteps = 1000
gaussian_loss_type = "mse"
scheduler = "cosine"

[train.main]
steps = 3000
lr = 0.001
weight_decay = 1e-5
batch_size = 1024

[train.T]
seed = 0
normalization = "quantile"
num_nan_policy = "__none__"
cat_nan_policy = "__none__"
cat_min_frequency = "__none__"
cat_encoding = "one-hot"
y_policy = "default"

[sample]
num_samples = 6497
batch_size = 2048
seed = 0

[eval.type]
eval_model = "none"
eval_type = "none"

[eval.T]
seed = 0
normalization = "quantile"
num_nan_policy = "__none__"
cat_nan_policy = "__none__"
cat_min_frequency = "__none__"
cat_encoding = "one-hot"
y_policy = "default"
"""

# Save to Google Drive
config_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine/config.toml"
with open(config_path, "w") as f:
    f.write(config_text)

config_path


'/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine/config.toml'

In [13]:
import sys
sys.path.append('/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main')



In [17]:
!pip uninstall -y skorch


Found existing installation: skorch 0.11.0
Uninstalling skorch-0.11.0:
  Successfully uninstalled skorch-0.11.0


In [18]:
!pip install skorch==0.13.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 16.2 MB/s eta 0:00:00


In [20]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder

path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine"

# Helper to convert one dataset split
def process_split(split_name):
    df = pd.read_csv(f"{path}/{split_name}.csv")

    X_num = df.drop(columns=["quality", "type"]).values.astype(np.float32)
    y = df["quality"].values.astype(np.float32)

    # Encode categorical column 'type'
    X_cat = df[["type"]].values
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_cat = enc.fit_transform(X_cat).astype(np.float32)

    np.save(f"{path}/X_num_{split_name}.npy", X_num)
    np.save(f"{path}/X_cat_{split_name}.npy", X_cat)
    np.save(f"{path}/y_{split_name}.npy", y)

for split in ["train", "val", "test"]:
    process_split(split)


In [24]:
import json
from pathlib import Path

info = {
    "target_col": "quality",
    "cat_features": ["type"],
    "num_features": [
        "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
        "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
        "pH", "sulphates", "alcohol"
    ],
    "normalization": "quantile",
    "cat_encoding": "one-hot",
    "y_policy": "default",
    "task_type": "regression"  # ✅ NEW LINE!
}

output_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine/info.json"
Path(output_path).write_text(json.dumps(info, indent=4))



484

In [27]:
file_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/lib/data.py"

text = open(file_path, "r").read()
text = text.replace("subsample=1e9", "subsample=int(1e9)")
open(file_path, "w").write(text)



25176

In [29]:
file_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/lib/data.py"

with open(file_path, "r") as f:
    lines = f.readlines()

# Replace the buggy lines around line 242–243
for i, line in enumerate(lines):
    if "nan_masks = {k: v == CAT_MISSING_VALUE for k, v in X.items()}" in line:
        lines[i] = "    nan_masks = {}\n"
        lines[i+1] = "    for k, v in X.items():\n        mask = np.array(v == CAT_MISSING_VALUE)\n        nan_masks[k] = mask\n"
    elif "if any(x.any() for x in nan_masks.values()):" in line:
        lines[i] = "    if any(mask.any() for mask in nan_masks.values()):\n"

with open(file_path, "w") as f:
    f.writelines(lines)


In [31]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine/train.csv")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4547 entries, 0 to 4546
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4547 non-null   float64
 1   volatile acidity      4547 non-null   float64
 2   citric acid           4547 non-null   float64
 3   residual sugar        4547 non-null   float64
 4   chlorides             4547 non-null   float64
 5   free sulfur dioxide   4547 non-null   float64
 6   total sulfur dioxide  4547 non-null   float64
 7   density               4547 non-null   float64
 8   pH                    4547 non-null   float64
 9   sulphates             4547 non-null   float64
 10  alcohol               4547 non-null   float64
 11  quality               4547 non-null   float64
 12  type                  4547 non-null   object 
dtypes: float64(12), object(1)
memory usage: 461.9+ KB


In [38]:
file_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/scripts/tune_evaluation_model.py"

with open(file_path, "r") as f:
    lines = f.readlines()

with open(file_path, "w") as f:
    for line in lines:
        if '"task_type": "CPU"' in line:
            f.write('        "task_type": "GPU",  # switched to GPU\n')
            f.write('        "devices": "0",      # specify GPU device\n')
        else:
            f.write(line)


In [42]:
file_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/scripts/pipeline.py"

with open(file_path, "r") as f:
    lines = f.readlines()

with open(file_path, "w") as f:
    skip = False
    for line in lines:
        if 'if config.eval.type == "catboost":' in line:
            skip = True
            f.write("# " + line)  # comment out the condition
            continue
        if skip:
            if line.strip().startswith(")"):
                skip = False
            f.write("# " + line)  # comment all lines in the block
        else:
            f.write(line)

print("✅ CatBoost block commented out in pipeline.py.")



✅ CatBoost block commented out in pipeline.py.


In [44]:
!PYTHONPATH="/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main" python scripts/pipeline.py --config data/wine/config.toml --train --sample --eval


/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/lib/data.py:244: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = np.array(v == CAT_MISSING_VALUE)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
[0]
16
{'is_y_cond': False, 'd_in': 16, 'num_classes': 0, 'rtdl_params': {'d_layers': [256, 256], 'dropout': 0.0}}
mlp
Step 500/3000 MLoss: 0.0 GLoss: 0.3315 Sum: 0.3315
Step 1000/3000 MLoss: 0.0 GLoss: 0.3236 Sum: 0.3236
Step 1500/3000 MLoss: 0.0 GLoss: 0.3169 Sum: 0.3169
Step 2000/3000 MLoss: 0.0 GLoss: 0.3147 Sum: 0.3147
Step 2500/3000 MLoss: 0.0 GLoss: 0.314 Sum: 0.314
Step 3000/3000 MLoss: 0.0 GLoss: 0.313 Sum: 0.313
/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/lib/data.py

In [47]:
import numpy as np
import pandas as pd

# Adjust path to your folder
base_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/exp/wine/check/"

X_num = np.load(base_path + "X_num_unnorm.npy")
X_cat = np.load(base_path + "X_cat_unnorm.npy")  # might be empty or all zeros if you have no cat
y = np.load(base_path + "y_train.npy")

print("X_num shape:", X_num.shape)
print("X_cat shape:", X_cat.shape)
print("y shape:", y.shape)


X_num shape: (6497, 12)
X_cat shape: (6497, 4)
y shape: (6497,)


In [48]:
# Just numerical
df_synth = pd.DataFrame(X_num)
df_synth["target"] = y

df_synth.head()


,0,1,2,3,4,5,6,7,8,9,10,11,target
0,-0.822387,-0.071880,1.231377,-1.762736,-0.884263,0.345575,-1.668880,-2.035580,0.199221,1.478781,0.637260,0.437007,-0.937230
1,0.835302,2.092062,0.398302,1.343011,-0.651713,0.862114,-0.969496,-1.351918,0.112805,-0.670246,0.301006,1.077311,1.048500
2,-1.209504,-0.683641,0.341067,-1.390445,-0.361143,1.067490,-1.666794,-1.521798,0.328310,1.834296,0.614481,0.702807,-0.937230
3,-1.630899,-0.373804,0.550158,0.624272,-0.550603,1.394979,2.174732,1.228960,-0.460198,-1.726635,-0.814165,-1.631338,-0.937230
4,-0.298895,-1.101830,0.564426,-1.084285,-0.307837,0.087013,-1.232933,-1.000866,-0.624142,1.196027,-1.012667,0.538419,0.207999


In [51]:
import numpy as np
import pandas as pd

base_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/exp/wine/check/"

X_num = np.load(base_path + "X_num_unnorm.npy")[:, :11]  # only first 11 columns
y = np.load(base_path + "y_train.npy")

num_columns = [
    "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
    "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
    "pH", "sulphates", "alcohol"
]

df_synth = pd.DataFrame(X_num, columns=num_columns)
df_synth["quality"] = y
df_synth.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.822387,-0.071880,1.231377,-1.762736,-0.884263,0.345575,-1.668880,-2.035580,0.199221,1.478781,0.637260,-0.937230
1,0.835302,2.092062,0.398302,1.343011,-0.651713,0.862114,-0.969496,-1.351918,0.112805,-0.670246,0.301006,1.048500
2,-1.209504,-0.683641,0.341067,-1.390445,-0.361143,1.067490,-1.666794,-1.521798,0.328310,1.834296,0.614481,-0.937230
3,-1.630899,-0.373804,0.550158,0.624272,-0.550603,1.394979,2.174732,1.228960,-0.460198,-1.726635,-0.814165,-0.937230
4,-0.298895,-1.101830,0.564426,-1.084285,-0.307837,0.087013,-1.232933,-1.000866,-0.624142,1.196027,-1.012667,0.207999


In [52]:
# Save to CSV
output_path = "/content/drive/MyDrive/TabDDPM/synthetic_wine_data.csv"
df_synth.to_csv(output_path, index=False)

print(f"Saved synthetic data to {output_path}")


Saved synthetic data to /content/drive/MyDrive/TabDDPM/synthetic_wine_data.csv


In [59]:
import os
import toml
import numpy as np
import pandas as pd
import subprocess

config_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine/config.toml"
pipeline_script = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/scripts/pipeline.py"
base_dir = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main"
exp_base = os.path.join(base_dir, "exp/wine")

# Numerical feature names
num_columns = [
    "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
    "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
    "pH", "sulphates", "alcohol"
]

for i in range(1, 5):
    # Step 1: Update config TOML
    config = toml.load(config_path)
    if "paths" not in config:
        config["paths"] = {}

    config["paths"]["save_path"] = f"exp/wine/check_{i}"

    with open(config_path, "w") as f:
        toml.dump(config, f)

    print(f"\n🔁 Running pipeline for iteration {i}...")

    try:
        result = subprocess.run(
            [
                "python", pipeline_script,
                "--config", config_path,
                "--train",
                "--sample",
                "--eval"
            ],
            env={**os.environ, "PYTHONPATH": base_dir},
            cwd=base_dir,
            check=True,
            capture_output=True,
            text=True,
        )
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"❌ Pipeline failed during iteration {i}")
        print("STDOUT:\n", e.stdout)
        print("STDERR:\n", e.stderr)
        break

        # Step 2: Load data if pipeline was successful
        folder = os.path.join(exp_base, f"check_{i}")
        X_num_path = os.path.join(folder, "X_num_unnorm.npy")
        y_path = os.path.join(folder, "y_train.npy")

        if not os.path.exists(X_num_path) or not os.path.exists(y_path):
            print(f"❌ Output files missing in {folder}. Skipping save.")
            continue

        X_num = np.load(X_num_path)
        y = np.load(y_path)

        # Step 3: Create and save CSV
        df = pd.DataFrame(X_num, columns=num_columns)
        df["quality"] = y
        df.to_csv(os.path.join(folder, f"wine_combined_{i}.csv"), index=False)
        print(f"✅ Saved wine_combined_{i}.csv")

    except subprocess.CalledProcessError as e:
        print("❌ Pipeline failed during iteration", i)
        print("STDOUT:\n", e.stdout)
        print("STDERR:\n", e.stderr)
        break

print("\n🎉 All 4 synthetic datasets are ready (if pipeline ran successfully)!")



🔁 Running pipeline for iteration 1...
Elapsed time: 0:00:00

❌ Output files missing in /content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/exp/wine/check_1. Skipping save.

🔁 Running pipeline for iteration 2...
Elapsed time: 0:00:00

❌ Output files missing in /content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/exp/wine/check_2. Skipping save.

🔁 Running pipeline for iteration 3...
Elapsed time: 0:00:00

❌ Output files missing in /content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/exp/wine/check_3. Skipping save.

🔁 Running pipeline for iteration 4...
Elapsed time: 0:00:00

❌ Output files missing in /content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/exp/wine/check_4. Skipping save.

🎉 All 4 synthetic datasets are ready (if pipeline ran successfully)!


In [61]:
import os
import toml
import numpy as np
import pandas as pd
import subprocess

config_path = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/data/wine/config.toml"
pipeline_script = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main/scripts/pipeline.py"
base_dir = "/content/drive/MyDrive/TabDDPM/tab-ddpm/tab-ddpm-main"
exp_base = os.path.join(base_dir, "exp/wine")

# Numerical feature names (excluding "type"!)
num_columns = [
    "fixed acidity", "volatile acidity", "citric acid", "residual sugar",
    "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density",
    "pH", "sulphates", "alcohol","type"
]

for i in range(1, 5):
    check_folder = f"exp/wine/check_{i}"

    # 🔧 Step 1: Update config
    config = toml.load(config_path)

    if "paths" not in config:
        config["paths"] = {}
    config["paths"]["save_path"] = check_folder

    # Ensure parent_dir is also set (needed by pipeline)
    config["parent_dir"] = check_folder

    # Save updated config
    with open(config_path, "w") as f:
        toml.dump(config, f)

    print(f"\n🔁 Running pipeline for iteration {i}...")

    try:
        result = subprocess.run(
            [
                "python", pipeline_script,
                "--config", config_path,
                "--train",
                "--sample",
                "--eval"
            ],
            env={**os.environ, "PYTHONPATH": base_dir},
            cwd=base_dir,
            check=True,
            capture_output=True,
            text=True,
        )
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"❌ Pipeline failed during iteration {i}")
        print("STDOUT:\n", e.stdout)
        print("STDERR:\n", e.stderr)
        break

    # 🔍 Step 2: Check and load outputs
    folder = os.path.join(base_dir, check_folder)
    X_num_path = os.path.join(folder, "X_num_unnorm.npy")
    y_path = os.path.join(folder, "y_train.npy")

    if os.path.exists(X_num_path) and os.path.exists(y_path):
        X_num = np.load(X_num_path)
        y = np.load(y_path)

        df = pd.DataFrame(X_num, columns=num_columns)
        df["quality"] = y

        out_csv = os.path.join(folder, f"wine_combined_{i}.csv")
        df.to_csv(out_csv, index=False)
        print(f"✅ Saved {out_csv}")
    else:
        print(f"❌ Output files missing in {folder}. Skipping save.")

print("\n🎉 All 4 synthetic datasets are ready (if pipeline ran successfully)!")


Streaming output truncated to the last 5000 lines.
Sample timestep  974
Sample timestep  973
Sample timestep  972
Sample timestep  971
Sample timestep  970
Sample timestep  969
Sample timestep  968
Sample timestep  967
Sample timestep  966
Sample timestep  965
Sample timestep  964
Sample timestep  963
Sample timestep  962
Sample timestep  961
Sample timestep  960
Sample timestep  959
Sample timestep  958
Sample timestep  957
Sample timestep  956
Sample timestep  955
Sample timestep  954
Sample timestep  953
Sample timestep  952
Sample timestep  951
Sample timestep  950
Sample timestep  949
Sample timestep  948
Sample timestep  947
Sample timestep  946
Sample timestep  945
Sample timestep  944
Sample timestep  943
Sample timestep  942
Sample timestep  941
Sample timestep  940
Sample timestep  939
Sample timestep  938
Sample timestep  937
Sample timestep  936
Sample timestep  935
Sample timestep  934
Sample timestep  933
Sample timestep  932
Sample timestep  931
Sample timestep  930
Samp